In [7]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

### 1-1

In [8]:
df = pd.read_csv('./Data/default.csv')
df['student'] = np.where(df['student']=='Yes', 1, 0)
df['default'] = np.where(df['default']=='Yes', 1, 0)
scaler = StandardScaler()
df[['balance', 'income']] = scaler.fit_transform( df[['balance', 'income']] )

df


,default,student,balance,income
0,0,0,-0.218835,0.813187
1,0,1,-0.037616,-1.605496
2,0,0,0.492410,-0.131212
3,0,0,-0.632893,0.164031
4,0,0,-0.102791,0.370915
...,...,...,...,...
9995,0,0,-0.255990,1.460366
9996,0,0,-0.160044,-1.039014
9997,0,0,0.020751,1.883565
9998,0,0,1.516742,0.236363


### 1-2

In [9]:
from sklearn.linear_model import LogisticRegression
X_train = df.to_numpy()[:, 1:]
y_train = df.to_numpy()[:,0]
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

LogisticRegression()

In [10]:
lr_clf.coef_, lr_clf.intercept_

(array([[-0.6017408 ,  2.73797718,  0.05238052]]), array([-5.93161678]))

### 1-3

$$ \mu\left(\text{student, balance, income}\right)\,=\,\sigma \left(-5.931617 \,-0.6017408\,\times\,\text{Student}\,+\,2.73797718\,\times\,\text{balance}\,+\,0.05238052\,\times\,\text{income}  \right) $$

### 1-4

In [31]:
def sigmoid(x):
    return 1/(1+np.exp(-x))


a = lr_clf.intercept_
b = lr_clf.coef_
coeffs = np.c_[a, b]

In [32]:
coeffs

array([[-5.93161678, -0.6017408 ,  2.73797718,  0.05238052]])

In [34]:
values = np.array([1, 900, 7100]).reshape(1,3)
new = np.c_[np.ones((1,1)), values]

new_pred_y = sigmoid(coeffs @ new.T)
new_pred_y

array([[1.]])

## 2

### 2-1

In [15]:
np.random.seed(123)
traindt = np.hstack( [np.ones((5,1)),\
                      np.around( np.random.randn(5,4), 3),
                      np.random.randint(1,4,(5,1))] )

traindt

array([[ 1.   , -1.086,  0.997,  0.283, -1.506,  2.   ],
       [ 1.   , -0.579,  1.651, -2.427, -0.429,  1.   ],
       [ 1.   ,  1.266, -0.867, -0.679, -0.095,  1.   ],
       [ 1.   ,  1.491, -0.639, -0.444, -0.434,  1.   ],
       [ 1.   ,  2.206,  2.187,  1.004,  0.386,  3.   ]])

In [146]:
X = traindt[:, :5]
y = traindt[:, 5]

In [155]:
theta_1 = np.vstack(([5, 2, 3, 1, 4],
                     [2, 4, 3, 1, 2],
                     [3, 4, 1, 5, 4]))

In [156]:
prob_1 = sigmoid(theta_1 @ X.T) / sigmoid(theta_1 @ X.T).sum(axis=0)
np.round(prob_1, 4)

array([[0.8151, 0.5549, 0.3353, 0.3312, 0.3333],
       [0.1739, 0.4451, 0.3332, 0.335 , 0.3333],
       [0.011 , 0.    , 0.3315, 0.3338, 0.3333]])

In [157]:
pred_y_1 = np.argmax(prob_1, axis=0)
pred_prob_1 = np.max(prob_1, axis=0)
pred_y_1, pred_prob_1

(array([0, 0, 0, 1, 2]),
 array([0.81509027, 0.55492431, 0.33533971, 0.33504894, 0.33333333]))

In [158]:
theta_2 = np.vstack([[5.5, 2. , 3. , 1.5, 4. ],
                     [2. , 3.5, 2.5, 1. , 1.5],
                     [3. , 4. , 1. , 5. , 4. ]])

prob_2 = sigmoid(theta_2 @ X.T) / sigmoid(theta_2 @ X.T).sum(axis=0)
np.round(prob_2, 4)

array([[0.7503, 0.571 , 0.3365, 0.3325, 0.3333],
       [0.2419, 0.429 , 0.3319, 0.3341, 0.3333],
       [0.0078, 0.    , 0.3316, 0.3334, 0.3333]])

In [159]:
pred_y_2 = np.argmax(prob_2, axis=0)
pred_prob_2 = np.max(prob_2, axis=0)
pred_y_2, pred_prob_2

(array([0, 0, 0, 1, 2]),
 array([0.75032521, 0.57097894, 0.33650474, 0.33405504, 0.33333334]))

In [160]:
pred_prob_1, pred_prob_2

(array([0.81509027, 0.55492431, 0.33533971, 0.33504894, 0.33333333]),
 array([0.75032521, 0.57097894, 0.33650474, 0.33405504, 0.33333334]))

In [167]:
np.where(y - (pred_y_1 + 1) == 0, 1, 0)

array([0, 1, 1, 0, 1])

In [173]:
- (np.where(y - (pred_y_1 + 1) == 0, 1, 0) @ np.log(pred_prob_1).T) / len(pred_prob_1)

0.5560294093669291

In [174]:
- (np.where(y - (pred_y_2 + 1) == 0, 1, 0) @ np.log(pred_prob_2).T) / len(pred_prob_2)

0.5496316521635999

In [175]:
'''로그손실 함수가 더 작은 2번째가 더 낫다'''

'로그손실 함수가 더 작은 2번째가 더 낫다'

In [181]:
def cross_entropy (y, y_pred, y_prob):

    return np.round((-1) * ( np.where(y - (y_pred + 1) == 0, 1, 0) @ np.log(y_prob).T ) / len(y), 6)

In [182]:
cross_entropy(y, pred_y_1, pred_prob_1), cross_entropy(y, pred_y_2, pred_prob_2)

(0.556029, 0.549632)

## 3